<a href="https://colab.research.google.com/github/Mochoye/Python/blob/master/ML/youtube/LicensePlateReader/License_Plate_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Mochoye/Licence-Plate-Detection-using-TensorFlow-Lite
!pip install easyocr

Cloning into 'Licence-Plate-Detection-using-TensorFlow-Lite'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 59 (delta 13), reused 7 (delta 2), pack-reused 0
Receiving objects: 100% (59/59), 55.52 MiB | 15.21 MiB/s, done.
Resolving deltas: 100% (13/13), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 20.2 MB/s eta 0:00:00


In [ ]:
import easyocr
import numpy as np
import cv2
import os
import sys
import glob
import random
import importlib.util
from tensorflow.lite.python.interpreter import Interpreter


reader = easyocr.Reader(['en'])

modelpath = '/content/Licence-Plate-Detection-using-TensorFlow-Lite/custom_model_lite/detect.tflite'
lblpath = '/content/Licence-Plate-Detection-using-TensorFlow-Lite/custom_model_lite/labelmap.txt'
min_conf = 0.5
input_video_path = '/content/Licence-Plate-Detection-using-TensorFlow-Lite/custom_model_lite/demo.mp4'
output_video_path = 'output.mp4'  # Change this to your desired output path

cap = cv2.VideoCapture(input_video_path)

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter(output_video_path, fourcc, 20.0, (frame_width, frame_height))

interpreter = Interpreter(model_path=modelpath)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
height = input_details[0]['shape'][1]
width = input_details[0]['shape'][2]

float_input = (input_details[0]['dtype'] == np.float32)

input_mean = 127.5
input_std = 127.5

crp_img = None  # Initialize variable to store the cropped image

with open(lblpath, 'r') as f:
    labels = [line.strip() for line in f.readlines()]

while True:
    ret, frame = cap.read()
    if not ret:
        break

    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    imH, imW, _ = frame.shape
    image_resized = cv2.resize(image_rgb, (width, height))
    input_data = np.expand_dims(image_resized, axis=0)

    # Normalize pixel values if using a floating model (i.e. if model is non-quantized)
    if float_input:
        input_data = (np.float32(input_data) - input_mean) / input_std

    # Perform the actual detection by running the model with the image as input
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()

    boxes = interpreter.get_tensor(output_details[1]['index'])[0]  # Bounding box coordinates of detected objects
    classes = interpreter.get_tensor(output_details[3]['index'])[0]  # Class index of detected objects
    scores = interpreter.get_tensor(output_details[0]['index'])[0]  # Confidence of detected objects

    detections = []

    for i in range(len(scores)):
        if (scores[i] > min_conf) and (scores[i] <= 1.0):
            # Get bounding box coordinates and draw box
            ymin = int(max(1, (boxes[i][0] * imH)))
            xmin = int(max(1, (boxes[i][1] * imW)))
            ymax = int(min(imH, (boxes[i][2] * imH)))
            xmax = int(min(imW, (boxes[i][3] * imW)))

            # Crop the detected region
            crp_img = frame[ymin:ymax, xmin:xmax]

            cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (10, 255, 0), 2)
            object_name = labels[int(classes[i])]  # Look up object name from "labels" array using class index
            label = '%s: %d%%' % (object_name, int(scores[i] * 100))  # Example: 'person: 72%'
            labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
            label_ymin = max(ymin, labelSize[1] + 10)

            result = reader.readtext(crp_img)
            if len(result) == 0:
              text = 'LicensePlate'
            else:
              text= result[0][-2]
              font = cv2.FONT_HERSHEY_SIMPLEX


            cv2.rectangle(frame, (xmin, label_ymin - labelSize[1] - 10), (xmin + labelSize[0], label_ymin + baseLine - 10), (255, 255, 255), cv2.FILLED)
            cv2.putText(frame, text, (xmin, label_ymin - 7), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
            detections.append([object_name, scores[i], xmin, ymin, xmax, ymax])



    out.write(frame)  # Write the frame to the output video

cap.release()
out.release()  # Release the output video
cv2.destroyAllWindows()

# The cropped image is now stored in the variable crp_img

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete